In [2]:
import tensorflow as tf 
from tensorflow import keras 
import matplotlib.pyplot as plt
import rasterio 
import raster_tools as rt 
import ai_tools as ai
import numpy as np
from joblib import dump

from keras.backend import clear_session
from numba import cuda

# Micro Feature Model

In [ ]:
input_dir = "training_data/micro_features/annoations"
target_dir =  "training_data/micro_features/annoations"
result_folder = "processed_raster"

In [ ]:
filters = 16
dropout= 0.15
batch_size= 64 * 2
img_size = (128,128)
nbands = 3
epochs = 250
nclasses =7
num_val_samples = 147

In [ ]:
train_generator, val_generator = ai.split_train_val(
    input_dir,
    target_dir,
    num_val_samples=num_val_samples,
    batch_size=batch_size,
    img_size=img_size,
    nbands=nbands, 
    nclasses=nclasses,
    augment=True
)

In [ ]:
input_img = keras.Input(img_size +(nbands,))
model = ai.get_multiUnet(
    input_img,
    nclasses=nclasses, 
    n_filters=filters, 
    dropout=dropout, 
    batchnorm=True
)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.categorical_crossentropy, 
    metrics="accuracy"
)

In [ ]:
history_micro = model.fit(
    train_generator,
    validation_data=val_generator, 
    batch_size=batch_size,
    epochs=epochs,
)

In [ ]:
dump(history_micro.history,"historys/history_micro_feature_training")
model.save("micro_feature_model")

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(8,4.5))

fig.suptitle("Model Performance - Micro-Features")

ax1.plot(history_micro['accuracy'])
ax1.plot(history_micro['val_accuracy'])

ax1.set_xlim(-1,150)

ax1.set_title('Accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epoch')
ax1.legend(['training', 'validation'], loc='lower right')


ax1.set_yticks([i for i in np.arange(0,1.1,0.1)])
ax1.set_ylim(0,1)
ax2.set_ylim(0,14)
ax2.set_yticks([i for i in np.arange(0,15,1)])

ax2.plot(history_micro['loss'])
ax2.plot(history_micro['val_loss'])
ax2.set_xlim(-1,150)
ax2.set_title('Loss')
ax2.set_ylabel('Accuracy')
ax2.set_xlabel('Epoch')
ax2.legend(['training', 'validation'], loc='upper right')
plt.savefig("model_low_res.png",dpi=300)

# Micro Features High Resolution (Drone Data)

In [ ]:
input_dir = "training_data/micro_features/images/"
target_dir =  "training_data/micro_features/annotations/"
result_folder = "ProcessedRaster/"

In [ ]:
train_generator, val_generator = ai.split_train_val(
    input_dir,
    target_dir,
    num_val_samples=10,
    batch_size=1,
    img_size=img_size,
    nbands=nbands, 
    nclasses=nclasses,
    augment=True

)

In [ ]:
model_drone = keras.models.load_model("micro_feature_model")

In [ ]:
hist_drone = model_drone.fit(
    train_generator, 
    validation_data=val_generator, 
    batch_size=1,
    epochs=200,
)

In [ ]:
model_drone.save("micro_feature_model_drone")
dump(hist_drone.history,"historys/history_micro_feature_training")

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(8,4.5))

ax1.plot(hist_drone.history['accuracy'])
ax1.plot(hist_drone.history['val_accuracy'])

ax1.set_xlim(-1,150)

ax1.set_title('Accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epoch')
ax1.legend(['training', 'validation'], loc='lower right')

ax2.plot(hist_drone.history['loss'])
ax2.plot(hist_drone.history['val_loss'])
ax2.set_xlim(-1,150)
ax2.set_title('Loss')
ax2.set_ylabel('Accuracy')
ax2.set_xlabel('Epoch')
ax2.legend(['train', 'vaidation'], loc='upper right')
plt.savefig("model_high_res.png",dpi=300)

# Prediction Micro Features

In [17]:
input_dir = "base_raster/"
ortho_fname = input_dir + "israelOrtho_20.tif"

In [ ]:
rast_src,rast = rt.readTiff(ortho_fname)

In [ ]:
nbands, nrows, ncols = rast.shape
b,r,c = rast.shape 
r += r % img_size[0]
c += r % img_size[1]
pad_rast = np.zeros((b,r,c),np.float32)
pad_rast[:,:nrows,:ncols] = rast

In [ ]:
prediction_func = ai.predict(model)
result = rt.mem_padTileIterator(rast,prediction_func,img_size,nclasses)[:,:nrows,:ncols]

In [ ]:
# from skimage.filters import gaussian
# result_smooth = gaussian(result,1.5)

In [ ]:
profile = rast_src.profile 
profile.update(dtype=np.float32, count=nclasses)
with rasterio.open(result_folder + "mirco_features_softmax.tif","w+",**profile) as dst:
    dst.write(result.astype(np.float32))


In [ ]:
profile = rast_src.profile 
profile.update(dtype=np.uint8, count=1)
with rasterio.open(result_folder + "micro_features_amax.tif","w+",**profile) as dst:
    dst.write(np.argmax(result,1).astype(np.uint8),1)


# Prediction Micro Features Drone Data

In [ ]:
full_img = input_dir + "Ortho_10cm.tif"
src = rasterio.open(full_img)
rast_src,rast = rt.memmap(full_img)
rast[:] = src.read()

In [ ]:
nbands, nrows, ncols = rast.shape

In [ ]:
b,r,c = rast.shape 
r += r % img_size[0]
c += r % img_size[1]
pad_rast =  rt.empty_memmap((b,r,c),np.float32)
pad_rast[:,:nrows,:ncols] = rast

In [ ]:
prediction_func = ai.predict(model_drone)
result = rt.empty_memmap((nclasses,nrows,ncols),np.float32)
result[:] = rt.mem_padTileIterator(pad_rast,prediction_func,img_size,nclasses)[:,:nrows,:ncols]

In [ ]:
profile = rast_src.profile 
profile.update(dtype=np.float32, count=nclasses)
with rasterio.open(result_folder + "micro_features_softmax_drone.tif","w+",**profile) as dst:
    dst.write(result.astype(np.float32))


In [ ]:
profile.update(dtype=np.uint8, count=1)
with rasterio.open(result_folder + f"micro_features_argmax_drone.tif","w+",**profile) as dst:
    dst.write(np.argmax(result,0).astype(np.uint8),1) 

# Cleanup

In [ ]:

clear_session()
cuda.select_device(0)
cuda.close()